In [1]:
pip install scikit-optimize

  Obtaining dependency information for scikit-optimize from https://files.pythonhosted.org/packages/65/cd/15c9ebea645cc9860aa71fe0474f4be981f10ed8e19e1fb0ef1027d4966e/scikit_optimize-0.10.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyaml>=16.9 from https://files.pythonhosted.org/packages/69/c1/ec1930bc6c01754b8baf3c99420f340b920561f0060bccbf81809db354cc/pyaml-25.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/107.8 kB ? eta -:--:--
   ---------------------------------------- 107.8/107.8 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch.optim as optim
from skopt import gp_minimize
from skopt.space import Integer, Categorical, Real
from lstm import AttentionLSTM


In [ ]:
# Define the objective function for Bayesian Optimization
def objective(params):
    num_layers = int(params[0])
    hidden_size = int(params[1])
    learning_rate = params[2]
    dropout_prob = params[3]
    optimizer_name = params[4]
    num_epochs = params[5]

    # Instantiate the CNN-LSTM model with these hyperparameters
    model = CNN_LSTM(input_size=3, hidden_size=hidden_size, num_layers=num_layers, num_classes=2, dropout_prob=dropout_prob).to(device)

    # Define the loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()

    # Define the optimizer
    if optimizer_name == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    # Training loop (for simplicity, use 1 epoch)
    model.train()
    num_epochs = num_epochs
    batch_size = 128

    for epoch in range(num_epochs):
        for i in range(0, len(X_train_tensor), batch_size):
            inputs = X_train_tensor[i:i+batch_size].to(device)
            labels = y_train_tensor[i:i+batch_size].to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

    # Validation step
    model.eval()
    with torch.inference_mode():
        val_outputs = model(X_test_tensor.to(device))
        val_loss = loss_fn(val_outputs, y_test_tensor.to(device))

    return val_loss.item()
